# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

## LLM Provider Configuration

You can switch between **Ollama** (local, free) and **OpenAI** (cloud API).

**To change:**
1. Edit the next cell
2. Change `USE_PROVIDER = 'ollama'` to `USE_PROVIDER = 'openai'`
3. Execute cells from here downward

### Cost Comparison:

| Provider | Model | Input Cost | Output Cost | Speed |
|----------|-------|------------|-------------|-------|
| **Ollama** | deepseek-v3.1:671b-cloud | FREE | FREE | Fast (local) |
| **OpenAI** | gpt-4o-mini | $0.15 / 1M tokens | $0.60 / 1M tokens | Very fast (cloud) |

**Estimate for this notebook:**
- With OpenAI: ~$0.01 - $0.05 per complete execution
- With Ollama: $0.00 (100% free)

In [2]:
# CONFIGURATION: Change here to test different providers
# Options: 'ollama' or 'openai'
USE_PROVIDER = 'ollama'  # Change to 'openai' to use OpenAI

print(f"Selected provider: {USE_PROVIDER.upper()}")

Selected provider: OLLAMA


In [3]:
# Initialize and constants

# Load configuration from global .env
load_dotenv(dotenv_path='/workspace/.env', override=True)

# Use provider selected in previous cell (or from .env as fallback)
llm_provider = USE_PROVIDER if 'USE_PROVIDER' in globals() else os.getenv('LLM_PROVIDER', 'ollama')

if llm_provider == 'ollama':
    # OLLAMA CONFIGURATION (Local)
    ollama_base_url = os.getenv('OLLAMA_BASE_URL')
    ollama_api_key = os.getenv('OLLAMA_API_KEY')
    ollama_model = os.getenv('OLLAMA_MODEL')
    
    print("=" * 50)
    print("Using OLLAMA (Local)")
    print("=" * 50)
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print(f"   Cost: FREE")
    print("=" * 50)
    
    # Create OpenAI client pointing to Ollama
    openai = OpenAI(
        base_url=f"{ollama_base_url}/v1",
        api_key=ollama_api_key
    )
    MODEL = ollama_model
    
else:
    # OPENAI CONFIGURATION (Cloud)
    api_key = os.getenv('OPENAI_API_KEY')
    
    print("=" * 50)
    print("Using OPENAI (Cloud API)")
    print("=" * 50)
    
    if api_key and api_key.startswith('sk-proj-') and len(api_key) > 50:
        print(f"   API Key: sk-proj-...{api_key[-8:]}")
        print(f"   Model: gpt-4o-mini")
        print(f"   Cost: ~$0.15 / 1M tokens input")
        print("   Status: Configured correctly")
    else:
        print("   Status: Invalid or missing API Key")
        print("   Check the .env file")
    
    print("=" * 50)
    
    openai = OpenAI(api_key=api_key)
    MODEL = 'gpt-4o-mini'

Using OLLAMA (Local)
   Base URL: http://192.168.80.200:11434
   Model: deepseek-v3.1:671b-cloud
   Cost: FREE


In [4]:
# Clase para representar una pagina web con BeautifulSoup

import requests
from bs4 import BeautifulSoup

class WebPage:
    """Class to represent a scraped webpage, with links and content."""
    
    def __init__(self, url):
        """
        Constructor: Downloads and parses a webpage.
        
        Args:
            url (str): URL of the webpage to scrape
        """
        self.url = url
        
        # 1. Make HTTP request and get content
        response = requests.get(url)
        self.body = response.content  # Fixed: was 'boby'
        
        # 2. Parse HTML with BeautifulSoup
        self.soup = BeautifulSoup(self.body, 'html.parser')  # Fixed: missing 'self.'
        
        # 3. Extract page title
        self.title = self.soup.title.string if self.soup.title else 'No Title'
        
        # 4. Clean and extract body text
        if self.soup.body:
            # Remove irrelevant elements (scripts, styles, navigation, etc.)
            for irrelevant in self.soup.body(['script', 'style', 'header', 'footer', 'nav', 'aside']):
                irrelevant.decompose()
            # Get all cleaned text
            self.text = self.soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = 'No Body Content'
        
        # 5. Extract all links (href) from page
        links = [link.get('href') for link in self.soup.find_all('a', href=True)]
        # Filter out empty or None links
        self.links = [link for link in links if link]
        
    def get_contents(self):
        """
        Returns page content in readable format.
        
        Returns:
            str: Title and text content of the page
        """
        return f"Title: {self.title}\n\nContent:\n{self.text}"

## Connection Test

Run this cell to verify that the selected provider works correctly.

In [5]:
# Quick connection test
print(f"Testing connection with {llm_provider.upper()}...\n")

try:
    test_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": "Respond only with: 'Connection successful!'"}
        ],
        max_tokens=50
    )
    result = test_response.choices[0].message.content
    print(f"SUCCESS!")
    print(f"Model response: {result}")
    print(f"Model used: {MODEL}")
except Exception as e:
    print(f"Connection ERROR:")
    print(f"   {str(e)}")

Testing connection with OLLAMA...

SUCCESS!
Model response: Connection successful!
Model used: deepseek-v3.1:671b-cloud


---

## First step: Have gpt-4o-mini or deepseek-v3.1:671b-cloud figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

IMPORTANT: You MUST respond with ONLY valid JSON, no other text before or after. Do not include markdown code blocks.

Respond in this exact JSON format:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)


You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

IMPORTANT: You MUST respond with ONLY valid JSON, no other text before or after. Do not include markdown code blocks.

Respond in this exact JSON format:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(webpage):
    """
    Generates user prompt with the list of links from a webpage.
    
    Args:
        webpage (WebPage): WebPage object with extracted links
    
    Returns:
        str: Formatted prompt with link list
    """
    user_prompt = f"""
Here is the list of links on the website {webpage.url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    # Use links directly from WebPage object (already extracted)
    user_prompt += "\n".join(webpage.links)
    return user_prompt

In [9]:
def select_relevant_links(webpage):
    """
    Selects relevant links from a webpage using GPT/Ollama.
    
    Args:
        webpage (WebPage): WebPage object with extracted links
    
    Returns:
        dict: JSON with relevant links in format {"links": [...]}
    """
    # NOTE: Ollama does not support response_format, so we make it conditional
    if llm_provider == 'ollama':
        # For Ollama: without response_format, rely on prompt
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(webpage)}
            ],
            temperature=0  # More deterministic for consistent JSON
        )
    else:
        # For OpenAI: use response_format
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(webpage)}
            ],
            response_format={"type": "json_object"}
        )
    
    result = response.choices[0].message.content
    
    # CLEANUP: Remove markdown code blocks if present
    # Some models wrap JSON in ```json ... ```
    if result.strip().startswith('```'):
        # Extract content between ``` markers
        lines = result.strip().split('\n')
        # Remove first line (```json or ```) and last line (```)
        result = '\n'.join(lines[1:-1])
    
    # Attempt to parse JSON with error handling
    try:
        links = json.loads(result)
        print(f"Found {len(links.get('links', []))} relevant links")
        return links
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON. Model response:")
        print(result)
        print(f"\nError: {e}")
        # Return empty structure on error
        return {"links": []}
    

In [10]:
antropic = WebPage("https://anthropic.com")
antropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/news/claude-sonnet-4-5',
 'https://www.anthropic.com/news/claude-haiku-4-5',
 'https://anthropic.com/news/context-management',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/core-views-on-ai-safety',
 'https://www.anthropic.com/rsp-updates',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/news/claude-haiku-4-5',
 'https://www.anthropic.com/news/claude-sonnet-4-5',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/news/claude-opus-4-1',
 'https://www.anthropic.com/research/project-vend-1',
 'https://www.anthropic.com/research/agentic-misalignment',
 'https://www.anthropic.com/research/tracing-thoughts-language-model',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/contact-sales']

In [11]:
select_relevant_links(antropic)

Found 10 relevant links


{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'},
  {'type': 'news page',
   'url': 'https://www.anthropic.com/news/claude-sonnet-4-5'},
  {'type': 'news page',
   'url': 'https://www.anthropic.com/news/claude-haiku-4-5'},
  {'type': 'news page',
   'url': 'https://anthropic.com/news/context-management'},
  {'type': 'product page', 'url': 'https://www.anthropic.com/claude/sonnet'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/project-vend-1'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/agentic-misalignment'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/tracing-thoughts-language-model'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/jobs'},
  {'type': 'contact page', 'url': 'https://www.anthropic.com/contact-sales'}]}

In [12]:
# Probar con HuggingFace
hf = WebPage("https://huggingface.co")
select_relevant_links(hf)

Found 7 relevant links


{'links': [{'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    """
    Fetches content from main page and all its relevant links.
    
    Args:
        url (str): Website URL
    
    Returns:
        str: Formatted content with main page and relevant links
    """
    # Create WebPage object (download and parse ONCE)
    webpage = WebPage(url)
    
    # Get main page content
    contents = webpage.get_contents()
    
    # Select relevant links using GPT
    relevant_links = select_relevant_links(webpage)
    
    # Build result
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Found 6 relevant links
## Landing Page:

Title: Hugging Face – The AI community building the future.

Content:
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Updated
6 days ago
•
810k
•
1.05k
Updated
4 days ago
•
19.4k
•
347
Updated
1 day ago
•
2.25M
•
2.46k
Updated
2 days ago
•
3.03k
•
221
Updated
about 14 hours ago
•
219
Browse 1M+ models
Spaces
1.38k
The Smol Training Playbook: The Secrets to Building World-Class LLMs
📝
15.7k
DeepSite v3
🐳
Generate any application by Vibe Coding
2.27k
Wan2.2 Animate
👁
Wan2.2 Animate
149
Dream-wan2-2-faster-Pro
🎬
Generate a video from an image with detailed prompts
2.05k
Wan2.2 14B Fast
🎥
generate a video from an image with a text prompt
Browse 400k+ applications
Datasets
Updated
7 days ago
•
12.4k
•
195
Updated
14 days ago
•
13.6k
•
207
Updated
Sep 9
•
4.53k
•
59
Updated
6 days ago
•
2.04k
•
4

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found 2 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nTitle: Hugging Face – The AI community building the future.\n\nContent:\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nUpdated\n6 days ago\n•\n810k\n•\n1.05k\nUpdated\n4 days ago\n•\n19.4k\n•\n347\nUpdated\n1 day ago\n•\n2.25M\n•\n2.46k\nUpdated\n2 days ago\n•\n3.03k\n•\n221\nUpdated\nabout 14 hours ago\n•\n219\nBrowse 1M+ models\nSpaces\n1.38k\nThe Smol Training Playbook: The Secrets to Building World-Class LLMs\n📝\n15.7k\nDeepSite v3\n🐳\nGenerate any application by Vibe Coding\n2.27k\nWan2.2 Animate\n👁\nWan2.2 Animate\n149\nDream-wan2-2-faster-Pro\n🎬\nGenerate a video from an image

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,  # FIXED: Use MODEL variable (gpt-4o-mini or ollama)
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found 7 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face – The AI Community Building the Future

## Who We Are

Hugging Face is the leading AI community and collaboration platform where developers, researchers, and organizations build the future of machine learning together. We provide the essential tools, infrastructure, and community space for creating, discovering, and collaborating on AI models, datasets, and applications across all modalities - including text, image, video, audio, and 3D.

## Our Platform & Ecosystem

**The Hub of Machine Learning Innovation**
- **1M+ models** available for exploration and deployment
- **400k+ applications** (Spaces) showcasing real-world AI implementations
- **250k+ datasets** for training and research
- Host and collaborate on unlimited public models, datasets, and applications

**Enterprise-Grade Solutions**
We provide Team and Enterprise subscriptions starting at $20/user/month, offering:
- Enterprise-grade security and access controls
- Single Sign-On integration
- Comprehensive audit logs and analytics
- Regional data management
- Private storage and datasets viewer
- Dedicated support and advanced compute options

## Trusted by Industry Leaders

More than 50,000 organizations trust Hugging Face, including major technology companies and research institutions:
- **Google** (34.5k followers, 1.05k models)
- **Microsoft** (16.3k followers, 426 models)
- **AI at Meta** (8.6k followers, 2.23k models)
- **Amazon**, **Intel**, **Grammarly**, **Writer**, and non-profits like **AI2**

## Open Source Foundation

We build the core ML tooling with the community through our popular open-source libraries:

**Transformers** (152k+ stars) - State-of-the-art AI models for PyTorch  
**Diffusers** (31k+ stars) - Advanced Diffusion models in PyTorch  
**Datasets** (20k+ stars) - Access & share datasets for any ML tasks  
**PEFT** (19k+ stars) - Parameter-efficient fine-tuning for large language models  
**TRL** (16k+ stars) - Train transformers with reinforcement learning  
**Transformers.js** (14k+ stars) - Run ML directly in your browser

## Why Join Hugging Face?

**For Developers & Researchers:**
- Build your ML portfolio by sharing work with the global community
- Accelerate development with our open-source stack
- Access 45,000+ models through a unified API with no service fees
- Deploy on optimized Inference Endpoints starting at $0.60/hour for GPU

**For Enterprises:**
- Scale your organization with the world's leading AI platform
- Benefit from enterprise-grade security and dedicated support
- Leverage advanced compute options like ZeroGPU with 5x quota boosts
- Gain comprehensive analytics and resource management tools

Hugging Face is more than a platform - we're a movement democratizing AI through collaboration, open-source innovation, and enterprise-ready solutions that empower everyone to build the future of machine learning.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,  # FIXED: Use MODEL variable (gpt-4o-mini or ollama)
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found 7 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face – The AI Community Building the Future

Hugging Face is the central platform where the global machine learning community collaborates to build the future of artificial intelligence. It is the home for creating, discovering, and collaborating on machine learning models, datasets, and applications.

## The Platform for ML Collaboration

*   **Host and Collaborate:** A platform for hosting and collaborating on unlimited public models, datasets, and applications.
*   **Vast Repository:** Access over **1 million models**, **250,000 datasets**, and **400,000 applications** across all modalities, including text, image, video, audio, and 3D.
*   **Build Your Portfolio:** A space for individuals and teams to share their work with the world and build their professional ML profiles.

## Enterprise-Grade Solutions

Hugging Face provides advanced tools for organizations of all sizes to accelerate their machine learning initiatives.

*   **Team & Enterprise Hub:** Offers enterprise-grade security, access controls, dedicated support, and single sign-on. Plans start at **$20/user/month**.
*   **Inference API:** Access over **45,000 models** from leading AI providers through a single, unified API with no service fees.
*   **Compute:** Deploy on optimized Inference Endpoints or upgrade your applications with GPU compute, starting at **$0.60/hour**.

## Trusted by Industry Leaders

More than **50,000 organizations** trust Hugging Face, including major technology innovators and enterprises such as:
*   **Google**, **Microsoft**, **AI at Meta**, **Amazon**, and **Intel**
*   Companies like **Grammarly** and **Writer**

## Open Source Leadership

Hugging Face is the foundation of modern ML tooling, building critical open-source libraries in collaboration with the community. Key projects include:
*   **Transformers:** State-of-the-art AI models for PyTorch.
*   **Diffusers:** Cutting-edge Diffusion models.
*   **Datasets:** Library for accessing and sharing datasets.
*   **Transformers.js:** Run state-of-the-art ML directly in your browser.

---
**Join the community that is building the future of AI.**

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found 7 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face: The AI Community Building the Future

## Company Overview
Hugging Face is the leading collaboration platform where the machine learning community works together on models, datasets, and applications. Positioned as "The Home of Machine Learning," Hugging Face enables creators to build, discover, and collaborate on AI development in an open, community-driven environment.

## Platform & Offerings

**Core Platform Features:**
- Hosting for unlimited public models, datasets, and applications
- Access to over 1 million AI models, 250k+ datasets, and 400k+ applications
- Support for multiple modalities including text, image, video, audio, and 3D
- Tools for building ML portfolios and sharing work globally

**Open Source Libraries:**
Hugging Face maintains essential open-source tools that form the foundation of modern AI development:
- **Transformers**: State-of-the-art AI models for PyTorch (152,088 stars)
- **Diffusers**: Advanced diffusion models in PyTorch
- **Datasets**: Access and share datasets for ML tasks
- **Transformers.js**: Run ML models directly in browsers
- **PEFT**: Parameter-efficient fine-tuning for large language models

**Enterprise Solutions:**
- Team & Enterprise plans starting at $20/user/month
- Enterprise-grade security with Single Sign-On and audit logs
- Dedicated support and resource groups
- Private datasets viewer and inference providers
- GPU compute starting at $0.60/hour

## Company Culture & Community
Hugging Face operates as a community-centric organization where collaboration and open-source development are foundational principles. The platform enables ML practitioners to build their professional profiles and portfolios while contributing to the broader AI ecosystem.

## Customers & Partnerships
More than 50,000 organizations trust Hugging Face, including major technology leaders:
- **AI2 (Allen Institute for AI)**: 805 models, 4.33k followers
- **Meta AI**: 2,200+ models, 8,600+ followers
- **Google**: 1,000+ models, 34,500+ followers
- **Microsoft**: 426 models, 16,300+ followers
- **Amazon, Intel, Grammarly, Writer**, and many others

## Career & Ecosystem Impact
While specific career information isn't provided, Hugging Face represents a vital hub for AI professionals to:
- Develop and showcase ML expertise
- Collaborate on cutting-edge AI projects
- Access state-of-the-art models and datasets
- Build professional networks within the AI community

Hugging Face has established itself as the central nervous system of the open AI ecosystem, empowering both individual developers and enterprise teams to accelerate machine learning development through community collaboration and robust platform tools.